In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import cleaned_econ_data

In [ ]:
STEPS = 5000
PRICE_NORM_FACTOR = 1000

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

"""Builds, trains, and evaluates the model."""
(train, test) = cleaned_econ_data.dataset()

# train_input_fn = tf.estimator.inputs.numpy_input_fn(
#     x={"x": np.array(train)},
#     y=np.array(training_set.target),
#     num_epochs=None,
#     shuffle=True)

# Build the training input_fn.
def input_train():
    return (
            # Shuffling with a buffer larger than the data set ensures
            # that the examples are well mixed.
            train.batch(128)
            # Repeat forever
            .repeat().make_one_shot_iterator().get_next())

# Build the validation input_fn.
def input_test():
    return (test.batch(128)
                    .make_one_shot_iterator().get_next())

feature_columns = [
        tf.feature_column.numeric_column(key="funds-rate"),
        tf.feature_column.numeric_column(key="oil-cpi"),
        tf.feature_column.numeric_column(key="high-tax-rate"),
        tf.feature_column.numeric_column(key="low-tax-rate"),
        tf.feature_column.numeric_column(key="unemployment"),
        tf.feature_column.numeric_column(key="snp")
]

# Build a DNNRegressor, with 2x20-unit hidden layers, with the feature columns
# defined above as input.
model = tf.estimator.DNNRegressor(
        hidden_units=[20, 20], feature_columns=feature_columns)

# Train the model.
model.train(input_fn=input_train, steps=STEPS)

# Evaluate how the model performs on data it has not yet seen.
# eval_result = model.evaluate(input_fn=input_test)
pred_result = list(model.predict(input_fn=input_test))


In [ ]:
predictions = [p['predictions'][0] for p in pred_result]
print(predictions)